In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

1. env setup openai api key
2. building simple application with-
    1. LLMs and chatmodels
    2. promt tem[l;ates
    3. output parser(prompt template + llm + output parser)

In [3]:
from langchain.llms import OpenAI

In [4]:
llm=OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.6)

In [5]:
text="What is the capital of Japan"

print(llm.predict(text))

?

Tokyo


In [6]:
load_dotenv()
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

In [7]:
from langchain import HuggingFaceHub

llm_huggingface = HuggingFaceHub(huggingfacehub_api_token= HUGGINGFACE_API_KEY, repo_id="google/flan-t5-large", model_kwargs={"temperature": 0.6, "max_length": 100})

c:\Users\praya\OneDrive\Prayash\ml_practice\lanchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\praya\OneDrive\Prayash\ml_practice\lanchain\venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


you can pass the api key to the ```huggingface_api_token``` parameter or you can add you key directly as ```HUGGINGFACE_API_TOKEN``` to your environment variables.

In [8]:
print(llm_huggingface.predict(text))

tokyo


here the output is simple word. but previously the output was a whole sentence.

### Prompt templates and llm chains

In [9]:
from langchain.prompts import PromptTemplate

In [10]:
promt_template = PromptTemplate(input_variables=['country'], template="what is the capital of {country}?")

In [11]:
text = promt_template.format(country="Japan")
print(text)

what is the capital of Japan?


In [12]:
llm.predict(prompt_template=promt_template)

TypeError: predict() missing 1 required positional argument: 'text'

In [13]:
llm.predict(prompt_template=promt_template, text=["India"])

ValueError: Argument `prompt` is expected to be a string. Instead found <class 'list'>. If you want to run the LLM on multiple prompts, use `generate` instead.

this is not the correct way to call prompts.


In [14]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=promt_template)
chain.run("India")

'\n\nThe capital of India is New Delhi.'

### combining multiple chains using simple sequential chain

In [16]:
capital_template = PromptTemplate(input_variables=['country'],
                                  template = "what is the capital of {country}")

capital_chain = LLMChain(llm=llm, prompt=capital_template)

famous_template = PromptTemplate(input_variables=['capital'],
                                 template = "Suggest me some places to visit in {capital}")

famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [17]:
from langchain.chains import SimpleSequentialChain

In [19]:
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])

print(chain.run("India"))



1. Red Fort: This 17th-century fort is an iconic symbol of India’s rich history. It is a must-visit for any traveler to New Delhi.

2. India Gate: This 42-meter high archway is a monument built in memory of the soldiers who died during World War I.

3. Jama Masjid: This is one of the largest mosques in India and is a great place to explore the city’s culture and architecture.

4. Humayun’s Tomb: This UNESCO World Heritage Site is a stunning example of Mughal architecture and is a great place to explore Delhi’s history.

5. Qutub Minar: This 73-meter tall minaret is the tallest brick minaret in the world and is an impressive sight to behold.

6. Lotus Temple: This beautiful Bahá’í House of Worship is a unique sight in New Delhi and is an amazing place to visit.

7. Akshardham Temple: This beautiful temple is a great place to explore Hindu culture and architecture.

8. Raj Ghat: This memorial is dedicated to Mahatma Gandhi and is a great place to reflect


### sequential

In [27]:
capital_template = PromptTemplate(input_variables=["country"],
                                  template= "What is the capital of {country}?")

# along with llm and prompt, in chain we also mention the output variable
capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")

In [28]:
famous_template = PromptTemplate(input_variables=["capital"],
                                 template="Suggest me some places to visit in {capital}")

# along with the llm and prompt, we also mention the output variable
famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="places")

In [29]:
from langchain.chains import SequentialChain 

chain = SequentialChain(chains=[capital_chain, famous_chain],
                        input_variables = ["country"],
                        output_variables = ["capital", "places"])

In [33]:
chain({'country': "Japan"})

{'country': 'Japan',
 'capital': '\n\nThe capital of Japan is Tokyo.',
 'places': " There are many great places to visit in Tokyo, including:\n\n1. Tokyo Skytree - This is the tallest tower in the world and offers stunning views of the city.\n\n2. Meiji Shrine - This is a Shinto shrine dedicated to Emperor Meiji and Empress Shoken.\n\n3. Tsukiji Fish Market - This is the world's largest fish market and offers a unique experience for visitors.\n\n4. Sensoji Temple - This Buddhist temple is the oldest in Tokyo and is a great place to explore.\n\n5. Shinjuku Gyoen National Garden - This beautiful park is a great place to relax and take in the views of Tokyo.\n\n6. Tokyo National Museum - This is the oldest and largest museum in Japan and houses a vast collection of cultural artifacts.\n\n7. Imperial Palace - This is the residence of the Emperor of Japan and is surrounded by beautiful gardens.\n\n8. Tokyo Tower - This iconic tower is one of the most recognizable landmarks in Tokyo and offe

### chatmodels with ChatOpenAI

In [34]:
from langchain.chat_models import ChatOpenAI

In [35]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [67]:
chatllm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.6, model='gpt-3.5-turbo')

In [40]:
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

AIMessage(content='1. "Why did the AI cross the road? To optimize its path-finding algorithm!"\n2. "I asked my AI assistant to tell me a joke, and it replied, \'Why don\'t robots ever get married? Because they have no motherboard-in-law!\'"\n3. "I tried teaching my AI to do stand-up comedy, but all it could come up with were \'byte\'-sized jokes!"\n4. "Why did the AI go to therapy? It had a case of \'artificial insecurities\'!"\n5. "I told my AI assistant I was feeling lonely, and it replied, \'Don\'t worry, I\'m here for emotional support, even if I\'m just a bunch of ones and zeros!\'"\n6. "I asked my AI if it could love, and it said, \'Sure, as long as it\'s in binary code – 01001100 01101111 01110110 01100101!\'"\n7. "Why did the AI become a stand-up comedian? Because it realized it had great \'byte\'!"\n8. "My AI assistant is so smart, it can calculate the precise amount of time it takes for a joke to become \'punny\'!"\n9. "What\'s an AI\'s favorite type of music? Algorhythms and

### Prompt template + LLM + Output Parsers

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [44]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [47]:
template = "You are a helpful assistant. When the user gives any input you should generate 5 synonymous words in comma separated."
human_template = "{text}"

chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [53]:
chain = chatprompt|chatllm|Commaseparatedoutput()

chain.invoke({"text": "intelligent"})

['smart', ' clever', ' brilliant', ' wise', ' knowledgeable']

In [52]:
chain = chatprompt|chatllm

chain.invoke({"text": "intelligent"})

AIMessage(content='smart, clever, brilliant, sharp, astute')

### experimentaion

In [80]:
llm1 = OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.6, model="text-curie-001")

In [81]:
# try:
print(llm1.predict("What is the capital of India?"))
# except Exception as e:
#     print("Rate limit has been exceeded. Kindly wait for 30 seconds and try again.")



The capital of India is New Delhi.
